In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('../src')

In [ ]:
from model.gap_resnet import GapResnet
from pathlib import Path
from PIL import Image
from torchvision import transforms
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
model_name = 'model_gap_resnet_6_train_acc=0.1073622.pth'
PATH = Path('../tensorboard_logs').joinpath(model_name)

In [ ]:
loc = 'cuda:0' if torch.cuda.is_available() else 'cpu'
model = torch.load(PATH, map_location=loc)

In [ ]:
test_images = list(Path('../data/test').glob('*.jpg'))

In [ ]:
def predict(img_path, n_test, top_k=5):
    img = Image.open(img_path).convert('RGB')
    test_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20, resample=Image.BILINEAR),
        transforms.RandomCrop((200, 800), pad_if_needed=True),
        transforms.ToTensor()
    ])
    
    with torch.no_grad():
        x = torch.stack([test_transform(img) for _ in range(n_test)])
        if torch.cuda.is_available():
            x = x.cuda()
            y = model(x).cpu().numpy()
        else:
            y = model(x).numpy()
    z = np.sum(y, axis=0)
#     for _ in range(n_test):
        
#         i = test_transform(img)
#         with torch.no_grad():
#             y = model(i[None]).numpy()
#             if z is None:
#                 z = y
#             else:
#                 z += y
    
    return  np.argsort(z.ravel())[-top_k:][::-1]

In [ ]:
with open('../data/labels.txt', 'r') as f:
    label_names = list(map(lambda x: x.strip(), f.readlines()))

In [ ]:
ids = []
for img_path in tqdm(test_images[:1]):
    preds = predict(img_path, 8)
    labels = [label_names[i] for i in preds]
    ids.append(' '.join(labels))

In [ ]:
image_files = [i.name for i in test_images]

In [ ]:
df = pd.DataFrame(columns=['Image', 'Id'])

In [ ]:
df['Image'] = image_files
df['Id'] = ids

In [ ]:
df

In [ ]:
df.to_csv(Path('submit').joinpath(Path(model_name).suffix))

In [ ]:
from PIL import Image
from argparse import ArgumentParser
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.nn.functional as F
from torch.optim import SGD
from torchvision import transforms
from tensorboardX import SummaryWriter
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss
from ignite.handlers import ModelCheckpoint

from data_loader import LabeledImageDataset
from model.gap_resnet import GapResnet

train_data_transform = transforms.Compose([
        transforms.RandomGrayscale(p=0.2),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20, resample=Image.BILINEAR),
        transforms.CenterCrop((200, 800)),
        transforms.ToTensor()
    ])

ds = LabeledImageDataset('../data', transform=train_data_transform)

pred = []
truth = []
for i in range(100):
    x, t = ds[i]
    y = predict(x, 8)
    #with torch.no_grad():
    #    y = np.argmax(model(x[None]).numpy())
    pred.append(y)
    truth.append(t)
pred = np.array(pred)
truth = np.array(truth)

In [ ]:
np.sum(pred == truth) / len(pred)

In [ ]:
df = pd.read_csv('../data/train_with_id.csv')

In [ ]:
ys = []
ls = []
for i in range(10):
    img, _, l = df.iloc[i]
    y = predict(Path('../data/train').joinpath(img), 8)
    ls.append(l)
    ys.append(y)
    
ys = np.array(ys)
ls = np.array(ls)

In [ ]:
np.sum(ys[:, 0]==ls) / len(ys)